In [1]:
import time 
import serial
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import showinfo
import modbus_crc

In [2]:
#初始化Marlin的串口
def initialize_Marlin_port():

    settings_window = tk.Toplevel(root)

    port=tk.StringVar()
    baudrate=tk.StringVar()
    
    port_label=ttk.Label(settings_window,text="port:")
    port_label.pack(fill="x",expand=True)
    port_entry=ttk.Entry(settings_window,textvariable=port)
    port_entry.pack(fill="x",expand=True)
    port_entry.focus()

    
    baudrate_label=ttk.Label(settings_window,text="baudrate:")
    baudrate_label.pack(fill="x",expand=True)
    baudrate_entry=ttk.Entry(settings_window,textvariable=baudrate)
    baudrate_entry.pack(fill="x",expand=True)
    baudrate_entry.focus()

    def start_to_initialize():
        global ser
        settings_window.destroy()
        port_=port.get()
        baudrate_=int(baudrate.get())
        ser=serial.Serial(port_,baudrate_)
        gcode ="G90"  # 启用绝对坐标
        ser.write(gcode.encode())
        showinfo(title="Information",message=f"初始化{'Marlin'}串口成功")
    #finish button
    finish_button=ttk.Button(settings_window,text="finish",command=start_to_initialize)
    finish_button.pack(fill='x', expand=True, pady=10)

In [3]:
#初始化Marlin的串口
def initialize_zhusheqi_port():

    settings_window = tk.Toplevel(root)

    port=tk.StringVar()
    baudrate=tk.StringVar()
    
    port_label=ttk.Label(settings_window,text="port:")
    port_label.pack(fill="x",expand=True)
    port_entry=ttk.Entry(settings_window,textvariable=port)
    port_entry.pack(fill="x",expand=True)
    port_entry.focus()

    
    baudrate_label=ttk.Label(settings_window,text="baudrate:")
    baudrate_label.pack(fill="x",expand=True)
    baudrate_entry=ttk.Entry(settings_window,textvariable=baudrate)
    baudrate_entry.pack(fill="x",expand=True)
    baudrate_entry.focus()
    
    def start_to_initialize():
        global se
        settings_window.destroy()
        port_=port.get()
        baudrate_=int(baudrate.get())
        se=serial.Serial(port_,baudrate_)
        showinfo(title="Information",message=f"初始化{'zhusheqi'}串口成功")
    #finish button
    finish_button=ttk.Button(settings_window,text="finish",command=start_to_initialize)
    finish_button.pack(fill='x', expand=True, pady=10)

In [4]:
def move_to_destination():
    time.sleep(1)
    settings_window = tk.Toplevel(root)
    x=tk.StringVar()
    y=tk.StringVar()
    z=tk.StringVar()

    x_label=ttk.Label(settings_window,text="x(mm):")
    x_label.pack(fill="x",expand=True)
    x_entry=ttk.Entry(settings_window,textvariable=x)
    x_entry.pack(fill="x",expand=True)
    x_entry.focus()

    
    y_label=ttk.Label(settings_window,text="y(mm):")
    y_label.pack(fill="x",expand=True)
    y_entry=ttk.Entry(settings_window,textvariable=y)
    y_entry.pack(fill="x",expand=True)
    y_entry.focus()

    z_label=ttk.Label(settings_window,text="z(mm):")
    z_label.pack(fill="x",expand=True)
    z_entry=ttk.Entry(settings_window,textvariable=z)
    z_entry.pack(fill="x",expand=True)
    z_entry.focus()

    def start_to_move():
        x_=x.get()
        y_=y.get()
        z_=z.get()
        ser.flushInput()
        data_to_send=f"G00 X{x_} Y{y_} Z{z_}\n"
        ser.write(data_to_send.encode())
    #finish button
    finish_button=ttk.Button(settings_window,text="finish",command=start_to_move)
    finish_button.pack(fill='x', expand=True, pady=10)

In [5]:
def define_zero_point():
    settings_window = tk.Toplevel(root)
    x=tk.StringVar()
    y=tk.StringVar()
    z=tk.StringVar()

    x_label=ttk.Label(settings_window,text="x(mm):")
    x_label.pack(fill="x",expand=True)
    x_entry=ttk.Entry(settings_window,textvariable=x)
    x_entry.pack(fill="x",expand=True)
    x_entry.focus()

    
    y_label=ttk.Label(settings_window,text="y(mm):")
    y_label.pack(fill="x",expand=True)
    y_entry=ttk.Entry(settings_window,textvariable=y)
    y_entry.pack(fill="x",expand=True)
    y_entry.focus()

    z_label=ttk.Label(settings_window,text="z(mm):")
    z_label.pack(fill="x",expand=True)
    z_entry=ttk.Entry(settings_window,textvariable=z)
    z_entry.pack(fill="x",expand=True)
    z_entry.focus()

    def set_zero_point():
        x_=x.get()
        y_=y.get()
        z_=z.get()
        ser.flushInput()
        data_to_send=f"G00 X{x_} Y{y_} Z{z_}\n"
        ser.write(data_to_send.encode())
    #finish button
    finish_button=ttk.Button(settings_window,text="finish",command=set_zero_point)
    finish_button.pack(fill='x', expand=True, pady=10)

In [6]:
def zhushe():
    settings_window = tk.Toplevel(root)
    V=tk.StringVar()
    V_label=ttk.Label(settings_window,text="移液体积(换算公式为1000=2.5mL):")
    V_label.pack(fill="x",expand=True)
    V_entry=ttk.Entry(settings_window,textvariable=V)
    V_entry.pack(fill="x",expand=True)
    V_entry.focus()
    def add_crc(data):
        data_to_send=data+bytes(modbus_crc.crc16(data))
        return data_to_send
    def transform_distance(distance,data_distance):
        data_distance.append(distance//256)
        data_distance.append(distance%256)
        data_distance_to_send=add_crc(bytes(data_distance))
        return data_distance_to_send
    def start_to_zhushe():
        se.flushInput()
        settings_window.destroy()
        distance_=int(V.get())
        data_distance=[0x03,0x06,0x00,0x07]
        data_distance_to_send=transform_distance(distance_,data_distance)
        se.write(data_distance_to_send)
        time.sleep(1)
        data_speed=[0x03,0x06,0x00,0x05,0x00,0xFF]
        data_speed_to_send=add_crc(bytes(data_speed))
        se.write(data_speed_to_send)
        time.sleep(1)
        data_fanzhuan=[0x03,0x06,0x00,0x01,0x00,0x01]
        data_fanzhuan_to_send=add_crc(bytes(data_fanzhuan))
        se.write(bytes(data_fanzhuan_to_send))
    #finish button
    finish_button=ttk.Button(settings_window,text="finish",command=start_to_zhushe)
    finish_button.pack(fill='x', expand=True, pady=10)

In [7]:
def chouqu():
    settings_window = tk.Toplevel(root)
    V=tk.StringVar()
    V_label=ttk.Label(settings_window,text="移液体积(换算公式为250=2mL):")
    V_label.pack(fill="x",expand=True)
    V_entry=ttk.Entry(settings_window,textvariable=V)
    V_entry.pack(fill="x",expand=True)
    V_entry.focus()
    def add_crc(data):
        data_to_send=data+bytes(modbus_crc.crc16(data))
        return data_to_send
    def transform_distance(distance,data_distance):
        data_distance.append(distance//256)
        data_distance.append(distance%256)
        data_distance_to_send=add_crc(bytes(data_distance))
        return data_distance_to_send
    def start_to_chouqu():
        se.flushInput()
        settings_window.destroy()
        distance_=int(V.get())
        data_distance=[0x03,0x06,0x00,0x07]
        data_distance_to_send=transform_distance(distance_,data_distance)
        se.write(data_distance_to_send)
        time.sleep(1)
        data_speed=[0x03,0x06,0x00,0x05,0x00,0xFF]
        data_speed_to_send=add_crc(bytes(data_speed))
        se.write(data_speed_to_send)
        time.sleep(1)
        data_zhengzhuan=[0x03,0x06,0x00,0x00,0x00,0x01]
        data_zhengzhuan_to_send=add_crc(bytes(data_zhengzhuan))
        se.write(bytes(data_zhengzhuan_to_send))
    #finish button
    finish_button=ttk.Button(settings_window,text="finish",command=start_to_chouqu)
    finish_button.pack(fill='x', expand=True, pady=10)

In [8]:
root=tk.Tk()
root.title("Controller")
#初始化串口的按钮
initialize_label=ttk.Label(root,text="初始化串口")
initialize_label.pack()
initialize_button=ttk.Button(root,text="初始化Marlin串口",command=initialize_Marlin_port)
initialize_button.pack(fill='x', expand=True, pady=10)
initialize_button=ttk.Button(root,text="初始化注射泵串口",command=initialize_zhusheqi_port)
initialize_button.pack(fill='x', expand=True, pady=10)

#操作按钮
control_label=ttk.Label(root,text="控制器")
control_label.pack()

control_label=ttk.Label(root,text="三轴移动平台")
control_label.pack()

#移动三轴
move_button=ttk.Button(root,text="三轴移动",command=move_to_destination)
move_button.pack(fill='x', expand=True, pady=10)

#设置初始点
initial_button=ttk.Button(root,text="回到初始点",command=define_zero_point)
initial_button.pack(fill='x', expand=True, pady=10)

#设置注射体积
zhushe_control_button=ttk.Button(root,text="设置抽取体积",command=chouqu)
zhushe_control_button.pack(fill='x', expand=True, pady=10)

#设置抽取体积
chouqu_control_button=ttk.Button(root,text="设置注射体积",command=zhushe)
chouqu_control_button.pack(fill='x', expand=True, pady=10)
root.mainloop()

